In [ ]:
# importing module
from pandas import *
from statistics import mean
import numpy as np
from helper760 import read_inputs
import tensorflow as tf

In [ ]:
# test case list 
# test_case 0 -> Use CT Data to detect cancer (Keras Neural Network)
# test_case 1 -> use CT_Data + Clinic Data to predict cancer (Keras Neural Network)
# test_case 2 -> Use CT Data to detect all 11 disease (Keras Neural Network)
# test_case 3 -> Use CT Data + Clinic Data to detect all 11 disease (Keras Neural Network)

test_case = [0,1,2,3]
n_split = 8
re_train_case_0 = True
re_train_case_1 = True
re_train_case_2 = True
re_train_case_3 = True



In [ ]:
Clininc_Data,Outcome_Data,CT_Data = read_inputs()

# Test Case 0 - Keras CT-To-Cancer Prediction
This is a test run perform prediction of cancer using CT Data. This test run uses a neural network constructed using Keras

In [ ]:
with tf.device("cpu:0"):
    if (0 in test_case):
        Clininc_Data,Outcome_Data,CT_Data = read_inputs()
        import numpy as np
        from tensorflow import keras
        from keras.models import Sequential
        from keras.layers import Dense
        from sklearn.preprocessing import StandardScaler 
        from sklearn.model_selection import KFold

        if re_train_case_0 == True:
            X = np.array(CT_Data)
            X = X.T
            X = X.astype(float)
            y = np.array(Outcome_Data[21])
            y = y.astype(int)
            standardScalar = StandardScaler()
            standardScalar.fit(X)
            standardScalar.transform(X)

            models_1 = []
            evas_1 = []
            counter = 0

            for train_index,test_index in KFold(n_split).split(X):
                print("fold "+str(counter))
                x_train,x_test = X[train_index,:], X[test_index,:]
                y_train,y_test = y[train_index], y[test_index]
                # define the keras model
                model = Sequential()
                model.add(Dense(32, input_dim=11, use_bias=True))
                model.add(Dense(64, activation='relu',use_bias=True))
                model.add(Dense(128, activation='relu',use_bias=True))
                model.add(Dense(256, activation='relu'))
                model.add(Dense(1, activation='sigmoid'))
                opt = keras.optimizers.Adam(learning_rate=0.0001)
                # compile the keras model
                model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
                # fit the keras model on the dataset
                model.fit(x_train, y_train, epochs=1000, batch_size=50,verbose=0)
                # evaluate the keras model
                _, accuracy,precision,recall = model.evaluate(x_test, y_test)   

                models_1.append(model)
                evas_1.append(accuracy)
                
                print(str(counter)+" fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
                    .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))
                counter = counter + 1
        
            model = models_1[evas_1.index(max(evas_1))]
            model.save("NN_models/keras_model_part1_case0")
        else:
            model = tf.keras.models.load_model("NN_models/keras_model_part1_case0", custom_objects=None, compile=True, options=None)


        # =========================================================
        from sklearn.metrics import precision_score
        from sklearn.metrics import recall_score
        from sklearn.metrics import f1_score
        test_y = model.predict(X)
        print(X.shape)
        print(test_y.shape)
        test_y = test_y > 0.5
        right_count = 0
        wrong_count = 0
        cancer_dectected = 0
        for i in range(len(test_y)):
            if test_y[i] == y[i]:
                right_count = right_count + 1
                if test_y[i] == True:
                    cancer_dectected = cancer_dectected + 1
            else:
                wrong_count = wrong_count + 1
        print(right_count)
        print(wrong_count)
        print(cancer_dectected)
        print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected cancer: "+str(cancer_dectected))
        print("precision score is: {0:.4%}".format(precision_score(y, test_y)))
        print("recall_score is: {0:.4%}".format(recall_score(y, test_y)))
        print("f1_score: {0:.4%}".format(f1_score(y, test_y)))

# Test Case 1 - NN CT+Clinic-To-Cancer Prediction
In this test run we tried to use CT_Data + Clinic Data to predict cancer

In [ ]:
with tf.device("cpu:0"):
    if (1 in test_case):
        Clininc_Data,Outcome_Data,CT_Data = read_inputs()
        import numpy as np
        from tensorflow import keras
        from keras.models import Sequential
        from keras.layers import Dense
        from sklearn.preprocessing import StandardScaler 
        from sklearn.model_selection import KFold

        if re_train_case_1 == True:
            X1 = np.array(CT_Data)
            X1 = X1.T
            X1 = X1.astype(float)
            print(X1.shape)

            X2 = np.array(Clininc_Data)
            X2 = X2[4:14,:]
            X2 = X2.T
            X2 = X2.astype(float)
            print(X2.shape)
        
            X = np.column_stack((X1,X2))

            print(X.shape)



            y = np.array(Outcome_Data[21])
            y = y.astype(int)
            standardScalar = StandardScaler()
            standardScalar.fit(X)
            standardScalar.transform(X)

            models_1 = []
            evas_1 = []
            counter = 0

            for train_index,test_index in KFold(n_split).split(X):
                print("fold "+str(counter))
                x_train,x_test = X[train_index,:], X[test_index,:]
                y_train,y_test = y[train_index], y[test_index]
                # define the keras model
                model = Sequential()
                model.add(Dense(32, input_dim=21, use_bias=True))
                model.add(Dense(64, activation='relu',use_bias=True))
                model.add(Dense(128, activation='relu',use_bias=True))
                model.add(Dense(256, activation='relu'))
                model.add(Dense(1, activation='sigmoid'))
                opt = keras.optimizers.Adam(learning_rate=0.0001)
                # compile the keras model
                model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
                # fit the keras model on the dataset
                model.fit(x_train, y_train, epochs=1000, batch_size=50,verbose=0)
                # evaluate the keras model
                _, accuracy,precision,recall = model.evaluate(x_test, y_test)   

                models_1.append(model)
                evas_1.append(accuracy)
                
                print(str(counter)+" fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
                    .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))
                counter = counter + 1
        
            model = models_1[evas_1.index(max(evas_1))]
            model.save("NN_models/keras_model_part1_case1")
        else:
            model = tf.keras.models.load_model("NN_models/keras_model_part1_case1", custom_objects=None, compile=True, options=None)


        # =========================================================
        from sklearn.metrics import precision_score
        from sklearn.metrics import recall_score
        from sklearn.metrics import f1_score
        test_y = model.predict(X)
        print(X.shape)
        print(test_y.shape)
        test_y = test_y > 0.5
        right_count = 0
        wrong_count = 0
        cancer_dectected = 0
        for i in range(len(test_y)):
            if test_y[i] == y[i]:
                right_count = right_count + 1
                if test_y[i] == True:
                    cancer_dectected = cancer_dectected + 1
            else:
                wrong_count = wrong_count + 1
        print(right_count)
        print(wrong_count)
        print(cancer_dectected)
        print("validation accuracy is: "+str(right_count / (right_count + wrong_count)) + ". Successfully detected cancer: "+str(cancer_dectected))
        print("precision score is: {0:.4%}".format(precision_score(y, test_y)))
        print("recall_score is: {0:.4%}".format(recall_score(y, test_y)))
        print("f1_score: {0:.4%}".format(f1_score(y, test_y)))

# Test Case 2 - Keras CT-To-AllOutcome Prediction
This is a test run perform prediction of all outcomes using (CT Data Only). This test run uses a neural network constructed using Keras

In [ ]:
with tf.device("cpu:0"):
    if (2 in test_case):
        Clininc_Data,Outcome_Data,CT_Data = read_inputs()
        import numpy as np
        from tensorflow import keras
        from keras.models import Sequential
        from keras.layers import Dense
        from sklearn.preprocessing import StandardScaler 
        from sklearn.model_selection import KFold

        if re_train_case_2 == True:
            X = np.array(CT_Data)
            X = X.T
            X = X.astype(float)

            y = np.array(Outcome_Data)
            y = y.T
            y = y.astype(int)

            # the predictions are:
            # Idx 1 -> CVD
            # Idx 3 -> Heart Failure
            # Idx 5 -> MI DX
            # Idx 7 -> Type 2 Diabetes
            # Idx 9 -> Femoral Neck Fracture
            # Idx 11 -> Unspec Femoral Fracture
            # Idx 13 -> Forearm Fracture
            # Idx 15 -> Humerus Fracture
            # Idx 17 -> Pathologic Fracture
            # Idx 19 -> Alzheimers
            # Idx 21 -> Cancer (weather the patience has or doesn't have, regardless of 1 or 2 types)
            y = y[:,[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]]

            models_1 = []
            evas_1 = []
            counter = 0

            for train_index,test_index in KFold(n_split).split(X):
                print("fold "+str(counter))
                x_train,x_test = X[train_index,:], X[test_index,:]
                y_train,y_test = y[train_index], y[test_index]
                # define the keras model
                model = Sequential()
                model.add(Dense(32, input_dim=11, use_bias=True))
                model.add(Dense(64, activation='relu',use_bias=True))
                model.add(Dense(128, activation='relu',use_bias=True))
                model.add(Dense(256, activation='relu'))
                model.add(Dense(11, activation='sigmoid'))
                opt = keras.optimizers.Adam(learning_rate=0.0001)
                # compile the keras model
                model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
                # fit the keras model on the dataset
                model.fit(x_train, y_train, epochs=1000, batch_size=50,verbose=0)
                # evaluate the keras model
                _, accuracy,precision,recall = model.evaluate(x_test, y_test)   

                models_1.append(model)
                evas_1.append(accuracy)
                
                print(str(counter)+" fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
                    .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))
                counter = counter + 1
        
            model = models_1[evas_1.index(max(evas_1))]
            model.save("NN_models/keras_model_part1_case2")
        else:
            model = tf.keras.models.load_model("NN_models/keras_model_part1_case2", custom_objects=None, compile=True, options=None)


# Test Case 3 - Keras CT+Clinic-To-AllOutcome Prediction
This is a test run perform prediction of all outcomes using CT Data + Clinic Data. This test run uses a neural network constructed using Keras

In [57]:
with tf.device("cpu:0"):
    if (3 in test_case):
        Clininc_Data,Outcome_Data,CT_Data = read_inputs()
        import numpy as np
        from tensorflow import keras
        from keras.models import Sequential
        from keras.layers import Dense
        from sklearn.preprocessing import StandardScaler 
        from sklearn.model_selection import KFold

        if re_train_case_3 == True:
            X1 = np.array(CT_Data)
            X1 = X1.T
            X1 = X1.astype(float)
            print(X1.shape)

            X2 = np.array(Clininc_Data)
            X2 = X2[4:14,:]
            X2 = X2.T
            X2 = X2.astype(float)
            print(X2.shape)
        
            X = np.column_stack((X1,X2))

            print(X.shape)

            y = np.array(Outcome_Data)
            y = y.T
            y = y.astype(int)

            # the predictions are:
            # Idx 1 -> CVD
            # Idx 3 -> Heart Failure
            # Idx 5 -> MI DX
            # Idx 7 -> Type 2 Diabetes
            # Idx 9 -> Femoral Neck Fracture
            # Idx 11 -> Unspec Femoral Fracture
            # Idx 13 -> Forearm Fracture
            # Idx 15 -> Humerus Fracture
            # Idx 17 -> Pathologic Fracture
            # Idx 19 -> Alzheimers
            # Idx 21 -> Cancer (weather the patience has or doesn't have, regardless of 1 or 2 types)
            y = y[:,[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]]

            models_1 = []
            evas_1 = []
            counter = 0

            for train_index,test_index in KFold(n_split).split(X):
                print("fold "+str(counter))
                x_train,x_test = X[train_index,:], X[test_index,:]
                y_train,y_test = y[train_index], y[test_index]
                # define the keras model
                model = Sequential()
                model.add(Dense(32, input_dim=21, use_bias=True))
                model.add(Dense(64, activation='relu',use_bias=True))
                model.add(Dense(128, activation='relu',use_bias=True))
                model.add(Dense(256, activation='relu'))
                model.add(Dense(11, activation='sigmoid'))
                opt = keras.optimizers.Adam(learning_rate=0.0001)
                # compile the keras model
                model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy','Precision','Recall'])
                # fit the keras model on the dataset
                model.fit(x_train, y_train, epochs=1000, batch_size=50,verbose=0)
                # evaluate the keras model
                _, accuracy,precision,recall = model.evaluate(x_test, y_test)   

                models_1.append(model)
                evas_1.append(accuracy)
                
                print(str(counter)+" fold average accuracy is: {0:.4%},average precision is: {1:.4%},average recall is: {2:.4%}" \
                    .format(np.mean(accuracy),np.mean(precision),np.mean(recall)))
                counter = counter + 1
        
            model = models_1[evas_1.index(max(evas_1))]
            model.save("NN_models/keras_model_part1_case3")
        else:
            model = tf.keras.models.load_model("NN_models/keras_model_part1_case3", custom_objects=None, compile=True, options=None)

(9223, 11)
(9223, 10)
(9223, 21)
fold 0


KeyboardInterrupt: 

if test_case == 1:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler

    param_grid = [
        {
            'weights': ['uniform'], 
            'n_neighbors': [i for i in range(1, 20)]
        },
        {
            'weights': ['distance'],
            'n_neighbors': [i for i in range(1, 20)], 
            'p': [i for i in range(1, 6)]
        }
    ]



    X = np.array(CT_Data)
    X = X.T
    X = X.astype(float)
    standardScalar = StandardScaler()
    standardScalar.fit(X)
    standardScalar.transform(X)
    y = np.array(Outcome_Data[21])
    y = y.astype(int)
    print(X.shape)
    print(y.shape)
    knn_clf = KNeighborsClassifier()
    grid_search = GridSearchCV(knn_clf, param_grid , cv = 5, scoring = 'average_precision')
    grid_search.fit(X, y)
    print(grid_search.best_estimator_)
    print(grid_search.best_score_)


    # ============================================
    def predict_Knn(test_vec, train_data, train_label, k):
        res_list = []
        dis_list = []
        train_data = train_data.astype(float)
        test_vec = test_vec.astype(float)
        for i in range(train_data.shape[1]):
            diff = (test_vec.reshape((train_data.shape[0],1))) - train_data[:,i]
            dis_list.append(np.linalg.norm(diff))

            
        # Smallest K elements indices
        # using sorted() + lambda + list slicing
        res = sorted(range(len(dis_list)), key = lambda sub: dis_list[sub])[:k] 

        for i in range(len(res)):
            res_list.append(train_label[res[i]])

        count_0 = 0
        count_1 = 0

        for i in range(len(res_list)):
            if res_list[i] == 0:
                count_0 = count_0 + 1
            else:
                count_1 = count_1 + 1

        #if count_1 > count_0:
        #    return 1
        #else:
        #    return 0        

        # as long as one nearest neighbour has cancer,
        # than we predict cancer
        if count_1 > 0:
            return 1
        else:
            return 0
    